In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat
from meltcurves import melt_fromdebris_func
from meltcurves import debris_frommelt_func
from spc_split_lists import split_list


debug=True
verbose=False

In [3]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [4]:
# Glaciers optimized
overwrite = False
# rois = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14', '15', '16','17','18']
rois = ['12','13','14','15','18']

# Percentiles
percentiles = [0.025, 0.05, 0.16, 0.25, 0.5, 0.75, 0.84, 0.95, 0.975]

# Uncertainty dataframe and dictionary for bounds
hd_uncertainty_fullfn = debris_prms.output_fp + 'hd_uncertainty_bnds_1std.csv'
hd_uncertainty_df = pd.read_csv(hd_uncertainty_fullfn)
hd_uncertainty_dict_low = dict(zip([int(np.round(x*100)) for x in hd_uncertainty_df['hd_m']], 
                                   list(hd_uncertainty_df['hd_bndlow_both'].values)))
hd_uncertainty_dict_low[0] = 0
hd_uncertainty_dict_low[1] = 0
hd_uncertainty_dict_high = dict(zip([int(np.round(x*100)) for x in hd_uncertainty_df['hd_m']], 
                                   list(hd_uncertainty_df['hd_bndhigh_both'].values)))
hd_uncertainty_dict_high[0] = hd_uncertainty_df.loc[0,'hd_bndhigh_both']
hd_uncertainty_dict_high[1] = hd_uncertainty_df.loc[0,'hd_bndhigh_both']


## ===== REGIONAL MELT FACTOR STATISTICS =====
for nroi, roi in enumerate(rois):
        
    print('roi:', roi)

    rgiids = []
    hd_fns = []
    # Filepaths
    if roi in ['13', '14', '15']:
        hd_fp = debris_prms.output_fp + 'ts_tif/hd_tifs/HMA/'
        hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/HMA/'
    else:
        hd_fp = debris_prms.output_fp + 'ts_tif/hd_tifs/' + roi + '/'
        hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi + '/'
    hd_fp_extrap = hd_fp + 'extrap/'
    hdopt_prms_fp_extrap = hdopt_prms_fp + '/_extrap/'
    mf_fp = hd_fp + 'meltfactor/'
    mf_fp_extrap = hd_fp_extrap + 'meltfactor/'

    # Glaciers optimized
    glac_hd_fullfns = []
    for i in os.listdir(hd_fp):
        if i.endswith('hdts_m.tif'):
            reg_str = str(int(i.split('.')[0])).zfill(2)
            if reg_str == roi:
                hd_fns.append(i)
                rgiids.append(i.split('_')[0])

    # Glaciers extrapolated
    for i in os.listdir(hd_fp_extrap):
        if i.endswith('hdts_m_extrap.tif'):
            reg_str = str(int(i.split('.')[0])).zfill(2)
            if reg_str == roi:
                hd_fns.append(i)
                rgiids.append(i.split('_')[0])

    # Sorted files        
    hd_fns = [x for _,x in sorted(zip(rgiids, hd_fns))]
    rgiids = sorted(rgiids)     

    main_glac_rgi = debris_prms.selectglaciersrgitable(rgiids)
    main_glac_rgi['CenLon_360'] = main_glac_rgi['CenLon']
    main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'] = (
        360 + main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'])
    main_glac_rgi['hd_fn'] = hd_fns

    for nglac, glac_idx in enumerate(main_glac_rgi.index.values):
#     for nglac, glac_idx in enumerate(main_glac_rgi.index.values[0:1]):
        glac_str = main_glac_rgi.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        if nglac%1000 == 0:
            print(nglac, glac_str)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'

        gf = create_glacfeat(thick_dir, thick_fn)

        # =====FILENAMES =====
        # Add the filenames
        fn_dict = OrderedDict()
        # DEM
        z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
        z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
        fn_dict['z1'] = z1_fp + z1_fn

        # Debris thickness and melt factors
        hd_fn = main_glac_rgi.loc[glac_idx, 'hd_fn']
        if '_extrap' not in hd_fn:
            hd_fullfn = hd_fp + hd_fn
            mf_fullfn = mf_fp + hd_fn.replace('hdts_m', 'meltfactor')
            hdopt_prms_fullfn = hdopt_prms_fp + glac_str_noleadzero + '_hdopt_prms.csv'
        else:
            hd_fullfn = hd_fp_extrap + hd_fn
            mf_fullfn = mf_fp_extrap + hd_fn.replace('hdts_m', 'meltfactor')
            hdopt_prms_fullfn = hdopt_prms_fp_extrap + glac_str + '_hdopt_prms_extrap.csv'

        fn_dict['debris_thick_ts'] = hd_fullfn
        fn_dict['meltfactor_ts'] = mf_fullfn

        # Ice thickness
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        fn_dict['ice_thick'] = thick_dir + thick_fn

        # ===== PROCESS THE DATA =====
        #Expand extent to include buffered region around glacier polygon
        warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
        if verbose:
            print("Expanding extent")
            print(gf.glac_geom_extent)
            print(warp_extent)
            print(gf.aea_srs)

        #Warp everything to common res/extent/proj
        z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
        z1_res = np.min([z1_gt[1], -z1_gt[5]])
        # resampling algorithm
        r_resampling = 'cubic'
        ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                           t_srs=gf.aea_srs, verbose=verbose, r=r_resampling)
        ds_dict = dict(zip(fn_dict.keys(), ds_list))
        gf.ds_dict = ds_dict

        if verbose:
            print(ds_list)
            print(fn_dict.keys())

        glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
        gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)

        # Debris thickness values of 0 are masked (use meltfactor mask instead)
        gf.meltfactor_ts = np.ma.array(iolib.ds_getma(ds_dict['meltfactor_ts']), mask=glac_geom_mask)
        gf.debris_thick_ts = np.ma.array(iolib.ds_getma(ds_dict['debris_thick_ts']), mask=glac_geom_mask)
        gf.debris_thick_ts = np.ma.array(gf.debris_thick_ts.data, mask=gf.meltfactor_ts.mask)

#             # Melt factors are masked so only calculate over areas with debris > 0
#             gf.debris_thick_ts = np.ma.array(iolib.ds_getma(ds_dict['debris_thick_ts']), mask=glac_geom_mask)
#             gf.meltfactor_ts = np.ma.array(iolib.ds_getma(ds_dict['meltfactor_ts']), mask=glac_geom_mask)       
#             gf.meltfactor_ts = np.ma.array(gf.meltfactor_ts.data, mask=gf.debris_thick_ts.mask)

        gf.res = geolib.get_res(ds_dict['z1'])

        if verbose:
            print('\n\n# z1 pixels:', gf.z1.count(), '\n')
            
        # ===== ADD UNCERTAINTY OF DEBRIS THICKNESS AND MELT FACTORS ===================================
        # ===== DEBRIS THICKNESS =====
        # Debris thickness (lower bound)
        hdts_bndlow = np.round(np.array(gf.debris_thick_ts.data)*100,0)
        hdts_bndlow_int = hdts_bndlow.astype(int).copy()
        for i in list(hd_uncertainty_dict_low.keys()):
            hdts_bndlow[hdts_bndlow_int == i] = hd_uncertainty_dict_low[i]
        gf.debris_thick_ts_bndlow = np.ma.array(hdts_bndlow, mask=gf.debris_thick_ts.mask)

        # Debris thickness (upper bound)
        hdts_bndhigh = np.round(np.array(gf.debris_thick_ts.data)*100,0)
        hdts_bndhigh_int = hdts_bndhigh.astype(int).copy()
        for i in list(hd_uncertainty_dict_high.keys()):
            hdts_bndhigh[hdts_bndhigh_int == i] = hd_uncertainty_dict_high[i]
        gf.debris_thick_ts_bndhigh = np.ma.array(hdts_bndhigh, mask=gf.debris_thick_ts.mask)
        
        # ===== MELT FACTORS =====
        # Optimized parameters for melt factor uncertainties
        df_opt = pd.read_csv(hdopt_prms_fullfn)
        melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
        melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
        func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]

        # Melt factor (lower bound)
        mf_array_low = melt_fromdebris_func(np.array(hdts_bndlow), func_coeff[0], func_coeff[1]) / melt_cleanice
        # limit melt rates to modeled 2 cm rate
        mf_array_low[mf_array_low > melt_2cm / melt_cleanice] = melt_2cm / melt_cleanice
        # Linearly interpolate between 0 cm and 2 cm for the melt rate
        def meltfactor_0to2cm_adjustment(mf, melt_clean, melt_2cm, hd):
            """ Linearly interpolate melt factors between 0 and 2 cm 
                based on clean ice and 2 cm sub-debris melt """
            mf = np.nan_to_num(mf,0)
            mf[(hd >= 0) & (hd < 0.02)] = (
                1 + hd[(hd >= 0) & (hd < 0.02)] / 0.02 * (melt_2cm - melt_clean) / melt_clean)
            return mf
        mf_array_low = meltfactor_0to2cm_adjustment(mf_array_low, melt_cleanice, melt_2cm, hdts_bndlow)
        gf.meltfactor_ts_bndlow = np.ma.array(mf_array_low, mask=gf.meltfactor_ts.mask)

        # Melt factor (lower bound)
        mf_array_high = melt_fromdebris_func(np.array(hdts_bndhigh), func_coeff[0], func_coeff[1]) / melt_cleanice
        mf_array_high[mf_array_high > melt_2cm / melt_cleanice] = melt_2cm / melt_cleanice
        mf_array_high = meltfactor_0to2cm_adjustment(mf_array_high, melt_cleanice, melt_2cm, np.array(hdts_bndhigh))
        gf.meltfactor_ts_bndhigh = np.ma.array(mf_array_high, mask=gf.meltfactor_ts.mask)
        
        
        # ===== PLOTS =====
        show_plots = False
        if debug and show_plots:
            # DEM
            var_full2plot = gf.z1.copy()
            clim = malib.calcperc(var_full2plot, (2,98))
            plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
            # Debris thickness
            var_full2plot = gf.debris_thick_ts.copy()
            clim = (0,1)
            plot_array(var_full2plot, clim, [gf.glacnum + ' hd (from ts)'], 'inferno', 'hd (m)', 
                       close_fig=False)
            # Melt factor
            var_full2plot = gf.meltfactor_ts.copy()
            clim = (0,1)
            plot_array(var_full2plot, clim, [gf.glacnum + ' meltfactor'], 'inferno', 'mf (-)',
                       close_fig=False)
            # Debris thickness lower bound
            var_full2plot = gf.debris_thick_ts_bndlow.copy()
            clim = malib.calcperc(var_full2plot, (2,98))
            plot_array(var_full2plot, clim, [glac_str + ' Hd_ts_bndlow'], 'inferno', 'hd (ma)', close_fig=False)
            # Debris thickness upper bound
            var_full2plot = gf.debris_thick_ts_bndhigh.copy()
            clim = malib.calcperc(var_full2plot, (2,98))
            plot_array(var_full2plot, clim, [glac_str + ' Hd_ts_bndlow'], 'inferno', 'hd (ma)', close_fig=False)
            # Melt factor lower bound
            var_full2plot = gf.meltfactor_ts_bndlow.copy()
            clim = (0,1)
            plot_array(var_full2plot, clim, [gf.glacnum + ' meltfactor-bndlow'], 'inferno', 'mf (-)', close_fig=False)
            # Melt factor upper bound
            var_full2plot = gf.meltfactor_ts_bndhigh.copy()
            clim = (0,1)
            plot_array(var_full2plot, clim, [gf.glacnum + ' meltfactor-bndhigh'], 'inferno', 'mf (-)', close_fig=False)
            
        # Bin data
        # ===== EXPORT THE BINNED DEBRIS THICKNESS =====
        gf.dc_area = None
        gf.ts = None
        outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
        if 'extrap' not in hd_fn:
            outbins_df_existing_fn = glac_str + '_mb_bins_hdts.csv'
            outbins_df_existing = pd.read_csv(debris_prms.mb_binned_fp_wdebris_hdts + outbins_df_existing_fn)
        else:
            outbins_df_existing_fn = glac_str + '_mb_bins_hdts_extrap.csv'
            outbins_df_existing = pd.read_csv(debris_prms.mb_binned_fp_wdebris_hdts + '../_wdebris_hdts_extrap/' + 
                                              outbins_df_existing_fn)
        outbins_df_existing['hd_ts_mean_m_bndlow'] = outbins_df['hd_ts_mean_m_bndlow']
        outbins_df_existing['hd_ts_std_m_bndlow'] = outbins_df['hd_ts_std_m_bndlow']
        outbins_df_existing['hd_ts_med_m_bndlow'] = outbins_df['hd_ts_med_m_bndlow']
        outbins_df_existing['hd_ts_mad_m_bndlow'] = outbins_df['hd_ts_mad_m_bndlow']
        outbins_df_existing['hd_ts_mean_m_bndhigh'] = outbins_df['hd_ts_mean_m_bndhigh']
        outbins_df_existing['hd_ts_std_m_bndhigh'] = outbins_df['hd_ts_std_m_bndhigh']
        outbins_df_existing['hd_ts_med_m_bndhigh'] = outbins_df['hd_ts_med_m_bndhigh']
        outbins_df_existing['hd_ts_mad_m_bndhigh'] = outbins_df['hd_ts_mad_m_bndhigh']
        outbins_df_existing['mf_ts_mean_bndlow'] = outbins_df['mf_ts_mean_bndlow']
        outbins_df_existing['mf_ts_std_bndlow'] = outbins_df['mf_ts_std_bndlow']
        outbins_df_existing['mf_ts_med_bndlow'] = outbins_df['mf_ts_med_bndlow']
        outbins_df_existing['mf_ts_mad_bndlow'] = outbins_df['mf_ts_mad_bndlow']
        outbins_df_existing['mf_ts_mean_bndhigh'] = outbins_df['mf_ts_mean_bndhigh']
        outbins_df_existing['mf_ts_std_bndhigh'] = outbins_df['mf_ts_std_bndhigh']
        outbins_df_existing['mf_ts_med_bndhigh'] = outbins_df['mf_ts_med_bndhigh']
        outbins_df_existing['mf_ts_mad_bndhigh'] = outbins_df['mf_ts_mad_bndhigh']
        # Output debris thickness
        output_df_new_fn = outbins_df_existing_fn.replace('.csv','_wbnds.csv')
        mb_bins_wbnds_fp = debris_prms.output_fp + 'mb_bins_wbnds/'
        if not os.path.exists(mb_bins_wbnds_fp):
            os.makedirs(mb_bins_wbnds_fp)
        outbins_df_existing.to_csv(mb_bins_wbnds_fp + output_df_new_fn, index=False)

roi: 12
1129 glaciers in region 12 are included in this model run: ['00002', '00004', '00005', '00006', '00007', '00008', '00010', '00011', '00012', '00013', '00014', '00016', '00018', '00019', '00021', '00023', '00024', '00026', '00031', '00032', '00034', '00036', '00037', '00038', '00040', '00041', '00044', '00045', '00047', '00049', '00050', '00051', '00052', '00053', '00057', '00059', '00061', '00062', '00063', '00064', '00065', '00066', '00067', '00068', '00069', '00073', '00074', '00075', '00079', '00080'] and more
This study is focusing on 1129 glaciers in region [12]
0 12.00002


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/ma/core.py:4008: RuntimeWarning: invalid value encountered in equal
  check = compare(sdata, odata)


1000 12.01639
roi: 13
8834 glaciers in region 13 are included in this model run: ['00067', '00080', '00093', '00137', '00175', '00188', '00190', '00198', '00199', '00200', '00202', '00203', '00209', '00210', '00211', '00217', '00218', '00221', '00222', '00223', '00226', '00228', '00233', '00235', '00236', '00240', '00246', '00248', '00250', '00299', '00335', '00345', '00358', '00386', '00391', '00394', '00399', '00400', '00402', '00413', '00420', '00500', '00502', '00503', '00505', '00507', '00508', '00515', '00516', '00517'] and more
This study is focusing on 8834 glaciers in region [13]
0 13.00067
1000 13.02609
2000 13.05927
3000 13.09317
4000 13.14815
5000 13.19073
6000 13.24347
7000 13.28708
8000 13.43528
roi: 14
5869 glaciers in region 14 are included in this model run: ['00005', '00018', '00020', '00026', '00028', '00029', '00032', '00033', '00036', '00043', '00056', '00057', '00063', '00065', '00072', '00075', '00079', '00097', '00101', '00104', '00122', '00127', '00131', '00142

In [24]:
np.where(main_glac_rgi.rgino_str == '11.00001')

(array([0]),)